# Extract features


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import pandas as pd
from math import pow, sqrt
from skimage.color import rgb2hsv 
from skimage import io 
from skimage.feature import local_binary_pattern
from google.colab import data_table
data_table.enable_dataframe_formatter()

def getEnergy(img):
    # Convert BGR to HSV and parse HSV
    hsv_img = rgb2hsv(img)
    h, s, v = hsv_img[:, :, 0], hsv_img[:, :, 1], hsv_img[:, :, 2]
    # Get height, width and number of pixels of the image
    height = img.shape[0]
    width = img.shape[1]
    pixels = (height-2) * (width-2)
    energy_h = [0] * pixels
    energy_s = [0] * pixels
    energy_v = [0] * pixels
    pixel = 0
        
    #Calculate the energy of each pixel (channel selected by user)
    #print('Getting energy from H channel')
    for j in range(height-2): #external for, height (start in j+1, end in height-2)
        for k in range(width-2): #internal for, width (start in k+1, end in width-2)
            x1 = h[j+1][k]
            x2 = h[j+1][k+2]
            y1 = h[j][k+1]
            y2 = h[j+2][k+1]
            
            energy = sqrt(pow((x1-x2),2) + pow((y1-y2),2))
            energy_h[pixel] = energy
            pixel += 1
            
    pixel = 0               
    #print('Getting energy from S channel')
    for j in range(height-2): #external for, height (start in j+1, end in height-2)
        for k in range(width-2): #internal for, width (start in k+1, end in width-2)
            x1 = s[j+1][k]
            x2 = s[j+1][k+2]
            y1 = s[j][k+1]
            y2 = s[j+2][k+1]
            
            energy = sqrt(pow((x1-x2),2) + pow((y1-y2),2))
            energy_s[pixel] = energy
            pixel += 1
            
    pixel = 0              
    #print('Getting energy from V channel')
    for j in range(height-2): #external for, height (start in j+1, end in height-2)
        for k in range(width-2): #internal for, width (start in k+1, end in width-2)
            x1 = v[j+1][k]
            x2 = v[j+1][k+2]
            y1 = v[j][k+1]
            y2 = v[j+2][k+1]
            
            energy = sqrt(pow((x1-x2),2) + pow((y1-y2),2))
            energy_v[pixel] = energy
            pixel += 1
    
    # get histogram from eHSV
    (histH, _) = np.histogram(energy_h)
    (histS, _) = np.histogram(energy_s)
    (histV, _) = np.histogram(energy_v)
    histHSV = np.hstack((histH,histS,histV))

    return histHSV #Array of 1 x 30

def lbphist(img,n_points, radius, METHOD):
  n_points = n_points - 2
  lbp = local_binary_pattern(img, n_points, radius, METHOD)
  # get histogram from LBP
  (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, n_points + 3),range=(0, n_points + 2))
  # normalize the histogram
  hist = hist.astype("float")
  hist /= (hist.sum() + 1e-7)   
  #print(hist.shape)
  return hist

def extract_features(path_folder,details):
    #print(path_folder)
    list_images = os.listdir(path_folder)
    #print(list_images)

  # Settings for Local Binary Pattern
    radius = 3
    n_points = 10
    METHOD = 'default'
    cont = 0
    for image in list_images:
      #print(name_image[-3:])
      if(image[-4:] == '.png'):
        #print(image)
        path_image = os.path.join(path_folder, image)
        img = io.imread(path_image, as_gray=False)
        img_gray = io.imread(path_image, as_gray=True)
        energy_HSV = getEnergy(img)
        hist = lbphist(img_gray,n_points, radius, METHOD)
        #details = ['SUR_AU','test','SUR','AU', 1]
        FOLDER = details[0]
        SET = details[1]
        VIEW = details[2]
        CLASS = details[3]
        LABEL = details[4]
        
        features = np.hstack((image, FOLDER, SET, VIEW, CLASS, LABEL, energy_HSV, hist))
        #print(features)
        if cont == 0:
          list_features = features
          cont += 1
        else:
          list_features = np.vstack((list_features,features))
    
    #return list_features

    df_features = pd.DataFrame(list_features, columns = ['image', 'folder', 'set', 'view', 'class','label','h0','h1','h2','h3','h4','h5','h6','h7','h8','h9','s0','s1','s2','s3','s4','s5','s6','s7','s8','s9','v0','v1','v2','v3','v4','v5','v6','v7','v8','v9','lbp0','lbp1','lbp2','lbp3','lbp4','lbp5','lbp6','lbp7','lbp8','lbp9'])
    return df_features

# Extract features (example)


In [ ]:
#Test:

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_AU/test/'
details = ['SEC_AU','test','SURFACE','AU',0]
test_AU = extract_features(path_data,details)
test_AU

,image,folder,set,view,class,label,h0,h1,h2,h3,...,lbp0,lbp1,lbp2,lbp3,lbp4,lbp5,lbp6,lbp7,lbp8,lbp9
0,10237 - section 1 - 10.png,SEC_AU,test,SURFACE,AU,0,15593,55,37,10,...,0.3731343283390049,0.10396294389583309,0.042202779205239176,0.08852290272318461,0.07874420998050724,0.008234688625412523,0.09624292330950886,0.14976839937469025,0.043232115283415744,0.01595470921173676
1,10237 - section 1 - 4.png,SEC_AU,test,SURFACE,AU,0,15535,112,37,18,...,0.35258358660827843,0.12158054710630291,0.042046605874263086,0.09017223910384133,0.07700101316732517,0.009118541032972717,0.08865248226501253,0.163627152980566,0.04154002026132016,0.013677811549459077
2,10237 - section 1 - 14.png,SEC_AU,test,SURFACE,AU,0,15273,186,60,27,...,0.3678217821600088,0.07673267326352808,0.058910891086192525,0.07623762375860209,0.10445544553938339,0.007425742573889814,0.09851485148027153,0.15594059405168612,0.04158415841378296,0.012376237623149692
3,10237 - section 1 - 12.png,SEC_AU,test,SURFACE,AU,0,15063,207,104,36,...,0.3570029382783054,0.09843290890801014,0.04162585700089981,0.09255631733141252,0.0945151811902784,0.01175318315319524,0.08129285014293375,0.17580803133321213,0.03476983349486925,0.012242899117911708
4,10237 - section 1 - 6.png,SEC_AU,test,SURFACE,AU,0,15285,157,51,27,...,0.3419999999829,0.11899999999404999,0.035499999998225,0.0899999999955,0.0779999999961,0.011499999999425,0.09349999999532499,0.180499999990975,0.039499999998025,0.010499999999474999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,8765 - section 3 - 8.png,SEC_AU,test,SURFACE,AU,0,15674,54,44,28,...,0.333830104297032,0.06929955290094637,0.0327868852434585,0.056631892693246505,0.12518628911138702,0.023845007449788,0.0685543964181405,0.23397913560104475,0.043219076002740754,0.012667660207699875
196,8765 - section 3 - 19.png,SEC_AU,test,SURFACE,AU,0,12741,2393,452,114,...,0.2646331409509783,0.16158285241866588,0.03380049463859847,0.1459192085617544,0.059356966194611956,0.004122011541292497,0.10469909314882941,0.19785655398203983,0.020610057706462483,0.0074196207743264945
197,8765 - section 2 - 19.png,SEC_AU,test,SURFACE,AU,0,15572,1,0,0,...,0.33798882679990744,0.09869646182035864,0.04283054003524998,0.08752327746333691,0.0945065176864755,0.005121042830301628,0.0982309124721494,0.1913407821139972,0.03910614524957607,0.004655493482092389
198,8765 - section 3 - 1.png,SEC_AU,test,SURFACE,AU,0,15656,9,7,26,...,0.3306930692851027,0.1438943894294459,0.03498349834752584,0.13069306929830407,0.08514851484586478,0.017821782177041466,0.06270627062292368,0.15577557754747356,0.02970297029506911,0.008580858085242187


# Extract features (dataset)

## Section

### Test

In [ ]:
dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_AU/test/'
details = ['SEC_AU','test','SECTION','AU',0]
test_AU = extract_features(path_data,details)
test_AU

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_BRU/test/'
details = ['SEC_BRU','test','SECTION','BRU',1]
test_BRU = extract_features(path_data,details)
test_BRU

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_CYS/test/'
details = ['SEC_CYS','test','SECTION','CYS',2]
test_CYS = extract_features(path_data,details)
test_CYS

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_STR/test/'
details = ['SEC_STR','test','SECTION','STR',3]
test_STR = extract_features(path_data,details)
test_STR

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_WD/test/'
details = ['SEC_WD','test','SECTION','WD',4]
test_WD = extract_features(path_data,details)
test_WD

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_WW/test/'
details = ['SEC_WW','test','SECTION','WW',5]
test_WW = extract_features(path_data,details)
test_WW

SEC_test = pd.concat([test_AU,test_BRU,test_CYS,test_STR,test_WD,test_WW], ignore_index=True)
SEC_test


,image,folder,set,view,class,label,h0,h1,h2,h3,...,lbp0,lbp1,lbp2,lbp3,lbp4,lbp5,lbp6,lbp7,lbp8,lbp9
0,10237 - section 1 - 10.png,SEC_AU,test,SECTION,AU,0,15593,55,37,10,...,0.3731343283390049,0.10396294389583309,0.042202779205239176,0.08852290272318461,0.07874420998050724,0.008234688625412523,0.09624292330950886,0.14976839937469025,0.043232115283415744,0.01595470921173676
1,10237 - section 1 - 4.png,SEC_AU,test,SECTION,AU,0,15535,112,37,18,...,0.35258358660827843,0.12158054710630291,0.042046605874263086,0.09017223910384133,0.07700101316732517,0.009118541032972717,0.08865248226501253,0.163627152980566,0.04154002026132016,0.013677811549459077
2,10237 - section 1 - 14.png,SEC_AU,test,SECTION,AU,0,15273,186,60,27,...,0.3678217821600088,0.07673267326352808,0.058910891086192525,0.07623762375860209,0.10445544553938339,0.007425742573889814,0.09851485148027153,0.15594059405168612,0.04158415841378296,0.012376237623149692
3,10237 - section 1 - 12.png,SEC_AU,test,SECTION,AU,0,15063,207,104,36,...,0.3570029382783054,0.09843290890801014,0.04162585700089981,0.09255631733141252,0.0945151811902784,0.01175318315319524,0.08129285014293375,0.17580803133321213,0.03476983349486925,0.012242899117911708
4,10237 - section 1 - 6.png,SEC_AU,test,SECTION,AU,0,15285,157,51,27,...,0.3419999999829,0.11899999999404999,0.035499999998225,0.0899999999955,0.0779999999961,0.011499999999425,0.09349999999532499,0.180499999990975,0.039499999998025,0.010499999999474999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,6536 - section - 36.png,SEC_WW,test,SECTION,WW,5,15822,14,0,0,...,0.32624113473249167,0.07151300235983965,0.04314420803527516,0.09101654845797774,0.06028368793969954,0.008274231677997977,0.10283687942654628,0.26063829785693626,0.03309692671199191,0.0029550827421421346
1196,6536 - section - 13.png,SEC_WW,test,SECTION,WW,5,15379,81,20,10,...,0.31569115813725457,0.13387297633039397,0.024906600247515155,0.10336239102718789,0.10087173100243638,0.009339975092818182,0.07471980074254546,0.20174346200487275,0.029887920297018186,0.00560398505569091
1197,6536 - section - 30.png,SEC_WW,test,SECTION,WW,5,14219,438,29,34,...,0.3272394880970603,0.1096892138872828,0.03960999390374101,0.07678244972109796,0.1212675197976071,0.02010968921266851,0.07556368067790592,0.19500304691072498,0.028031687993416717,0.006703229737556171
1198,6536 - section - 15.png,SEC_WW,test,SECTION,WW,5,14515,339,14,51,...,0.32976190474227607,0.12916666665897816,0.03630952380736253,0.09642857142283162,0.07321428570992772,0.007142857142431972,0.08035714285235969,0.2035714285593112,0.03452380952175453,0.00952380952324263


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


### Train

In [ ]:
dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_AU/train/'
details = ['SEC_AU','train','SECTION','AU',0]
train_AU = extract_features(path_data,details)
train_AU

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_BRU/train/'
details = ['SEC_BRU','train','SECTION','BRU',1]
train_BRU = extract_features(path_data,details)
train_BRU

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_CYS/train/'
details = ['SEC_CYS','train','SECTION','CYS',2]
train_CYS = extract_features(path_data,details)
train_CYS

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_STR/train/'
details = ['SEC_STR','train','SECTION','STR',3]
train_STR = extract_features(path_data,details)
train_STR

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_WD/train/'
details = ['SEC_WD','train','SECTION','WD',4]
train_WD = extract_features(path_data,details)
train_WD

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SEC_WW/train/'
details = ['SEC_WW','train','SECTION','WW',5]
train_WW = extract_features(path_data,details)
train_WW

SEC_train = pd.concat([train_AU,train_BRU,train_CYS,train_STR,train_WD,train_WW], ignore_index=True)
SEC_train

,image,folder,set,view,class,label,h0,h1,h2,h3,...,lbp0,lbp1,lbp2,lbp3,lbp4,lbp5,lbp6,lbp7,lbp8,lbp9
0,11459 - section 3 - 10.png,SEC_AU,train,SECTION,AU,0,15465,74,71,20,...,0.3528077141036411,0.12138400453083471,0.018718094156624045,0.0794100964220414,0.1117413499653011,0.01985252410551035,0.07146908677983725,0.1684628474096164,0.046511627904338534,0.009642654565533598
1,11459 - section 3 - 9.png,SEC_AU,train,SECTION,AU,0,15426,75,71,20,...,0.360946745542714,0.13501882731925666,0.032275416889065336,0.062399139318859646,0.13017751478589684,0.024206562666799,0.06670252823740169,0.1490048413045183,0.025820333511252267,0.013448090370443888
2,11459 - section 3 - 2.png,SEC_AU,train,SECTION,AU,0,15844,0,0,0,...,0.4027777777544689,0.1383101851771811,0.030671296294521335,0.07986111110648951,0.08796296295787251,0.017361111110106416,0.055555555552340534,0.1336805555478194,0.03356481481287241,0.020254629628457487
3,11459 - section 3 - 3.png,SEC_AU,train,SECTION,AU,0,15844,0,0,0,...,0.3680911680701942,0.13048433047689548,0.02450142450002841,0.06552706552333179,0.10199430198849035,0.01823361823257928,0.07350427350008522,0.16980056979089456,0.033048433046549947,0.014814814813970666
4,11459 - section 3 - 7.png,SEC_AU,train,SECTION,AU,0,15734,5,0,0,...,0.3660762211290351,0.1465378421821504,0.03811057434041274,0.07514761137546175,0.09071390230323596,0.017176596885819826,0.05528717122623257,0.17713365538501696,0.019860440149229174,0.01395598496972861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,9996 - section 1 - 40.png,SEC_WW,train,SECTION,WW,5,8627,3272,2535,1046,...,0.34747145185712497,0.05220228384707981,0.04295812941582609,0.08537248504157843,0.10005437737356963,0.004893964110663732,0.0951604132629059,0.21642196844935171,0.04676454594634233,0.008700380641179967
4796,9996 - section 1 - 23.png,SEC_WW,train,SECTION,WW,5,10616,3939,867,278,...,0.32328308205900147,0.09994416526521807,0.04466778335875668,0.127861529864441,0.05025125627860127,0.003350083751906751,0.11055276381292278,0.20938023449417195,0.025683975431285092,0.005025125627860127
4797,9996 - section 1 - 20.png,SEC_WW,train,SECTION,WW,5,15796,0,0,0,...,0.3609958506036828,0.08506224065948848,0.03267634854602301,0.10684647302350381,0.10477178422693093,0.005186721991432224,0.11670124480722503,0.14263485476438614,0.04201244813060101,0.003112033194859334
4798,9996 - section 1 - 16.png,SEC_WW,train,SECTION,WW,5,8106,3383,2457,1293,...,0.33665105384445837,0.0954332552637334,0.0532786885214708,0.13114754097592812,0.0790398126417424,0.0029274004682126814,0.0936768149828058,0.17681498828004596,0.024590163932986523,0.006440281030067899


## Surface

### Test

In [ ]:
dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_AU/test/'
details = ['SUR_AU','test','SURFACE','AU',0]
test_AU = extract_features(path_data,details)
test_AU

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_BRU/test/'
details = ['SUR_BRU','test','SURFACE','BRU',1]
test_BRU = extract_features(path_data,details)
test_BRU

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_CYS/test/'
details = ['SUR_CYS','test','SURFACE','CYS',2]
test_CYS = extract_features(path_data,details)
test_CYS

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_STR/test/'
details = ['SUR_STR','test','SURFACE','STR',3]
test_STR = extract_features(path_data,details)
test_STR

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_WD/test/'
details = ['SUR_WD','test','SURFACE','WD',4]
test_WD = extract_features(path_data,details)
test_WD

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_WW/test/'
details = ['SUR_WW','test','SURFACE','WW',5]
test_WW = extract_features(path_data,details)
test_WW

SUR_test = pd.concat([test_AU,test_BRU,test_CYS,test_STR,test_WD,test_WW], ignore_index=True)
SUR_test

,image,folder,set,view,class,label,h0,h1,h2,h3,...,lbp0,lbp1,lbp2,lbp3,lbp4,lbp5,lbp6,lbp7,lbp8,lbp9
0,4070 - surface 1 - 1.png,SUR_AU,test,SURFACE,AU,0,15800,3,0,0,...,0.3614730877982168,0.1246458923442127,0.028895184134340213,0.10991501415807846,0.1002832861132984,0.01189801699649303,0.07308781869274289,0.1303116147234951,0.04702549574804388,0.012464589234421269
1,4070 - surface 1 - 7.png,SUR_AU,test,SURFACE,AU,0,15752,7,8,2,...,0.3375251508884545,0.13078470824291827,0.044265593559141565,0.11066398389785392,0.0749496981853647,0.007042253520772522,0.0980885311821887,0.15492957745699548,0.029678068408969915,0.01207243460703861
2,4070 - surface 2 - 15.png,SUR_AU,test,SURFACE,AU,0,7169,6615,1682,243,...,0.21778741864564916,0.05770065075671581,0.02386117136555917,0.037310195226147064,0.1561822125745691,0.029934924076792414,0.12060737526591726,0.3067245119172788,0.036008676788025655,0.013882863339961698
3,4070 - surface 2 - 5.png,SUR_AU,test,SURFACE,AU,0,4611,6489,3237,1065,...,0.2580987785311684,0.08603292617705613,0.03186404673224301,0.0711630377020094,0.0934678704145795,0.024960169940257026,0.08072225172168229,0.3138608603125937,0.024960169940257026,0.01486988847504674
4,4070 - surface 1 - 15.png,SUR_AU,test,SURFACE,AU,0,15821,7,8,2,...,0.31728880155612527,0.12278978388394941,0.03732809430072062,0.09970530451376693,0.07367387033036965,0.0068762278975011675,0.1051080550046607,0.1920432219944969,0.031925343809826846,0.013261296659466537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,8234 - surface 2 - 11.png,SUR_WW,test,SURFACE,WW,5,10551,1744,1972,1097,...,0.39353348727974446,0.07852193995018375,0.03325635103772488,0.09053117782491774,0.1053117782861288,0.010161662817082603,0.0882217090028535,0.15658198613595464,0.03464203233096342,0.009237875288256911
1196,8234 - surface 2 - 13.png,SUR_WW,test,SURFACE,WW,5,7271,3225,942,2383,...,0.38841737391969955,0.057413879178361764,0.03295057413714675,0.09635546679498974,0.07039440838390441,0.010484273589092147,0.09685471791827983,0.19720419369959039,0.042935596602948795,0.006989515726061431
1197,8234 - surface 2 - 12.png,SUR_WW,test,SURFACE,WW,5,8269,3211,921,2071,...,0.3879472692849366,0.07627118643708705,0.03719397363290047,0.10499058379920008,0.075329566851444,0.0075329566851444,0.09698681732123415,0.17749529189371493,0.0282485875692915,0.008003766477965924
1198,8234 - surface 2 - 8.png,SUR_WW,test,SURFACE,WW,5,8663,2215,2947,1466,...,0.3507825148218682,0.05396654074182587,0.04101457096378766,0.11063140852074303,0.07393416081630144,0.007015650296437363,0.11171073933557955,0.2039935240041018,0.03777657851927811,0.009174311926110398


### Train

In [ ]:
dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_AU/train/'
details = ['SUR_AU','train','SURFACE','AU',0]
train_AU = extract_features(path_data,details)
train_AU

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_BRU/train/'
details = ['SUR_BRU','train','SURFACE','BRU',1]
train_BRU = extract_features(path_data,details)
train_BRU

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_CYS/train/'
details = ['SUR_CYS','train','SURFACE','CYS',2]
train_CYS = extract_features(path_data,details)
train_CYS

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_STR/train/'
details = ['SUR_STR','train','SURFACE','STR',3]
train_STR = extract_features(path_data,details)
train_STR

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_WD/train/'
details = ['SUR_WD','train','SURFACE','WD',4]
train_WD = extract_features(path_data,details)
train_WD

dir_root = '/content/drive/MyDrive/Proyecto_CalculosRenales/Imagenes/Patch/Yonathan Elbeze/Jonathan El Beze (patch - 6classes - w128 - 1k per class and view)/'
path_data = dir_root + 'SUR_WW/train/'
details = ['SUR_WW','train','SURFACE','WW',5]
train_WW = extract_features(path_data,details)
train_WW

SUR_train = pd.concat([train_AU,train_BRU,train_CYS,train_STR,train_WD,train_WW], ignore_index=True)
SUR_train

,image,folder,set,view,class,label,h0,h1,h2,h3,...,lbp0,lbp1,lbp2,lbp3,lbp4,lbp5,lbp6,lbp7,lbp8,lbp9
0,10237 - surface 3 - 1.png,SUR_AU,train,SURFACE,AU,0,15856,0,0,0,...,0.30780243376465266,0.17895490335154224,0.03149606298987143,0.13027916963992275,0.06871868288699222,0.022906227628997404,0.05869720829930585,0.16392269147001268,0.022906227628997404,0.014316392268123378
1,10237 - surface 2 - 3.png,SUR_AU,train,SURFACE,AU,0,13976,930,74,357,...,0.38736263733603277,0.14697802196792734,0.023351648350044527,0.08447802197221992,0.15865384614294958,0.0748626373574957,0.024038461536810544,0.07829670329132578,0.010989010988256248,0.010989010988256248
2,10237 - surface 2 - 13.png,SUR_AU,train,SURFACE,AU,0,15610,13,6,2,...,0.2672268907413318,0.09187675069513294,0.0291316526594324,0.08627450979908825,0.12492997198179663,0.0364145658242905,0.04593837534756647,0.29635854340076423,0.01400560224011173,0.007843137254462569
3,10237 - surface 1 - 11.png,SUR_AU,train,SURFACE,AU,0,15050,131,48,23,...,0.3800448430280244,0.09585201793184686,0.025784753361783364,0.07399103138598705,0.1205156950605092,0.022421524662420318,0.08688340806687873,0.15134529147133713,0.034753363226751495,0.008408071748407618
4,10237 - surface 2 - 9.png,SUR_AU,train,SURFACE,AU,0,12033,1512,649,145,...,0.33279656466275914,0.11540526032660196,0.02576489532872974,0.050993022004777615,0.21685453568347532,0.08427267847105353,0.03220611916091218,0.11755233493732944,0.011272141706319261,0.01288244766436487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,9996 - surface 1 - 15.png,SUR_WW,train,SURFACE,WW,5,15184,29,4,1,...,0.33186705765728897,0.10019550341641273,0.03323558162105398,0.11143695014118099,0.0869990224786413,0.006353861192260319,0.10117302052291431,0.19061583576780958,0.029814271748298423,0.008308895405263495
4796,9996 - surface 2 - 15.png,SUR_WW,train,SURFACE,WW,5,8872,3138,2635,780,...,0.3892183288199882,0.08948787061512195,0.03611859838080223,0.13045822101722596,0.07870619945667352,0.005929919137146635,0.0808625336883632,0.15040431266035556,0.03396226414911255,0.004851752021301792
4797,9996 - surface 2 - 14.png,SUR_WW,train,SURFACE,WW,5,9642,1802,1923,1715,...,0.3469094393700571,0.08864398657936541,0.04599904168442746,0.1130809774742175,0.09966459031625949,0.006708193578979004,0.07714422615825854,0.18926689026405047,0.02635361763170323,0.006229036894766218
4798,9996 - surface 1 - 14.png,SUR_WW,train,SURFACE,WW,5,15694,3,0,0,...,0.34086242298044855,0.07597535933901564,0.03182751539877682,0.12371663243718087,0.0718685831585283,0.003080082135365499,0.07443531827133289,0.24794661189692266,0.026180698150606738,0.004106776180487332


# Save

In [ ]:
# For UMAPS

SEC = pd.concat([SEC_train, SEC_test], ignore_index=True)
SUR = pd.concat([SUR_train, SUR_test], ignore_index=True)

with pd.ExcelWriter('feat_umap.xlsx') as writer:
    SEC.to_excel(writer, sheet_name='section',index = False)
    SUR.to_excel(writer, sheet_name='surface',index = False)

In [ ]:
# For XGBOOST

with pd.ExcelWriter('feat_train.xlsx') as writer:
    SEC_train.to_excel(writer, sheet_name='section',index = False)
    SUR_train.to_excel(writer, sheet_name='surface',index = False)

with pd.ExcelWriter('feat_test.xlsx') as writer:
    SEC_test.to_excel(writer, sheet_name='section',index = False)
    SUR_test.to_excel(writer, sheet_name='surface',index = False)